# Using Spark to Load Data

In [2]:
import re
from pyspark.sql import Row
from pyspark.ml.feature import MinHashLSH, CountVectorizer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, avg, min, max, from_unixtime, unix_timestamp, date_format, first, udf, substring, element_at
from pyspark.sql.types import StringType,IntegerType
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import storage
# !pip install gcsfs --upgrade

import pandas as pd

In [3]:
spark = SparkSession.builder \
            .appName("sentiment-analysis") \
            .config("spark.driver.maxResultSize", "16g") \
            .config("spark.num.executors", 32) \
            .config("spark.executor.cores", 4) \
            .config("spark.executor.memory", "10g") \
            .config("spark.driver.memory", "8g") \
            .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
            .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 21:35:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.debug.maxToStringField", 2000)
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
print("Spark version:", spark.version)

Spark version: 3.5.1


In [5]:
sc = spark.sparkContext
print('Original spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

# Stop existing Spark environment
sc.stop()

# Waiting for the environment to stop
sleep_time = 30
print(f'Waiting for {sleep_time} seconds for the enviroment to stop...')
time.sleep(sleep_time)

# Applying new configuration and restarting Spark
conf = pyspark.SparkConf().setAll([('spark.driver.maxResultSize', '4g')])
sc = pyspark.SparkContext(conf=conf)

print('New spark.driver.maxResultSize: ' + sc._conf.get('spark.driver.maxResultSize'))

# Starting  Spark session with configs applied
spark = SparkSession(sc).builder.getOrCreate()


Original spark.driver.maxResultSize: 16g
Waiting for 30 seconds for the enviroment to stop...


NameError: name 'time' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from google.cloud import storage

In [ ]:
from pyspark.sql import SparkSession

# Create spark_session
spark = SparkSession.builder \
        .appName("stonk-go") \
        .config("spark.num.executors", 50) \
        .config("spark.executor.cores", 8) \
        .config("spark.executor.memory", "16g") \
        .config("spark.driver.memory", "16g") \
        .config("spark.driver.maxResultSize", "16g") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "512m") \
        .getOrCreate()
    
# Add "eagerEval.enabled" to beautify the way Spark DF is displayed
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

# Ensure using the right kernel
print("Spark version:", spark.version)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')

# List all files in given COS directory
def list_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))
        
# List all files in given COS directory
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns=['Name','Size'])

    blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df

# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

In [ ]:
#adsp-capstone-enique-data/stonkgo/reddit-data-bitcoin/filtered_data
data_bucket = "adsp-capstone-enique-data"
data_folder = "stonkgo/reddit-data-bitcoin/filtered_data"

data_reddit_cmt = 'gs://adsp-capstone-enique-data/stonkgo/reddit-data-bitcoin/filtered_data/reddit_comments.parquet'
data_reddit_sub = 'gs://adsp-capstone-enique-data/stonkgo/reddit-data-bitcoin/filtered_data/reddit_submissions.parquet'

#data_reddit_cmt2 = 'gcs/stonkgo/reddit-data-bitcoin/filtered_data/reddit_comments.parquet'

In [1]:
list_blobs_pd(data_bucket, data_folder)

NameError: name 'list_blobs_pd' is not defined

In [23]:
cmt_df = spark.read.parquet(data_reddit_cmt) 
cmt_df.limit(5)

24/05/04 19:44:57 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://adsp-capstone-enique-data/stonkgo/reddit-data-bitcoin/filtered_data/reddit_comments.parquet.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.s

Py4JJavaError: An error occurred while calling o45.parquet.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "gs"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(